# Кибериммунный подход к разработке.<br>Шаблоны безопасности.<br>Шаблон "Обфускация чувствительных данных"

## Об авторе 

Этот блокнот разработан для вас Сергеем Соболевым, sergey.p.sobolev@kaspersky.com

Больше информации о кибериммунном подходе можно найти на странице https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82

Подписывайтесь на телеграм-канал @learning_cyberimmunity (https://t.me/learning_cyberimmunity)

Обучающие видео на тему кибериммунного подхода вы можете найти на youtube канале https://www.youtube.com/@learning_cyberimmunity/

## Ключевые технологии

- FLASK (FLux Advanced Security Kernel) - простая демонстрация FLASK в другом блокноте, здесь используем наработки <ссылка>
- Обфускация или шифрование данных - например, конфиденциальность которых необходимо обеспечить

Обфускация позволяет сохранить конфиденциальность данных, даже в случае компрометации внутренних компонент, до тех пор, пока данные остаются в защищённом виде.

## Сценарий примера

1. Создадим три сущности (Обработчик данных - DataProcessor, База данных - DataBase и Файловая система - FileSystem) и монитор (Monitor), который будет контролировать их взаимодействие
2. Определим политики безопасности
3. Подготовим данные, которые будут содержать некритичную и критичную части, критичную часть обработчик данных должен будет обфусцировать перед отправкой в базу данных

- Каждая сущность в качестве интерфейса использует очереди сообщений, у каждой сущности есть своя «персональная» очередь, ассоциированная с ней
- DataProcessor, DataBase и FileSystem отправляют сообщения только в очередь Monitor
- Monitor проверяет сообщения на соответствие политикам безопасности, в случае положительного решения перенаправляет сообщение в очередь соответствующей сущности

![диаграмма](./images/architecture-policy.png?raw=true)

Очередь событий для монитора безопасности: все запросы от сущностей друг к другу должны отправляться только в неё

In [105]:
from multiprocessing import Queue
monitor_events_queue = Queue()

Зафиксируем формат сообщений

In [106]:
from dataclasses import dataclass


@dataclass
class Event:
    source: str              # отправитель
    destination: str         # получатель
    operation: str           # чего хочет (запрашиваемое действие)
    parameters: str          # с какими параметрами
    data: str or None = None # дополнительные данные

### Монитор безопасности

Ниже в методе _check_policies можно увидеть пример политики безопасности:

```python
if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "read_request" \
                and event.parameters == "table_orders" \
                and event.data is None:
            authorized = True
```            

в этом примере проверяется отправитель сообщения, получатель, запрашиваемая операция и даже параметры операции. Это довольно жёсткий вариант, очевидно, количество проверок можно уменьшить.

А пока это место для экспериментов, как можно из монитора безопасности заблокировать взаимодействие между сущностями.

In [107]:
from multiprocessing import Queue, Process
from multiprocessing.queues import Empty


# формат управляющих команд для монитора или других сущностей при необходимости
@dataclass
class ControlEvent:
    operation: str


# Класс, реализующий поведение монитора безопасности
class Monitor(Process):

    def __init__(self, events_q: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        self._events_q = events_q  # очередь событий для монитора (входящие сообщения)
        self._control_q = Queue()  # очередь управляющих команд (например, для остановки монитора)
        self._entity_queues = {}   # словарь очередей известных монитору сущностей
        self._force_quit = False   # флаг завершения работы монитора

    # регистрация очереди новой сущности
    def add_entity_queue(self, entity_id: str, queue: Queue):
        print(f"[{self.__class__.__name__}] регистрируем сущность {entity_id}")
        self._entity_queues[entity_id] = queue

    # проверка политик безопасности        
    def _check_policies(self, event):
        print(f'[{self.__class__.__name__}] обрабатываем событие {event}')

        # default deny: всё, что не разрешено, запрещено по умолчанию!
        authorized = False

        # проверка на входе, что это экземпляр класса Event, 
        # т.е. имеет ожидаемый формат
        if not isinstance(event, Event):
            return False

        # 
        #  политики безопасности
        #

        # запрос на чтение данных из базы для обработки
        if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "read_request" \
                and event.parameters == "table_orders" \
                and event.data is None:
            authorized = True

        # ответ на запрос на чтение данных из базы для обработки
        # event.data не проверяем, т.к. может быть пустым или нет в 
        # зависимости от наличия данных
        if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "read_response" \
                and event.parameters == "table_orders":
            authorized = True            

        # запрос на запись обработанных данных в базу 
        if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "write_request" \
                and event.parameters == "table_orders" \
                and event.data is not None:
            authorized = True

        # запрос на запись данных в файловую систему
        if event.source == "DataBase" \
                and event.destination == "FileSystem" \
                and event.operation == "write_request" \
                and event.parameters is not None \
                and event.data is not None:
            authorized = True

        # запрос на чтение данных из файловой систему
        if event.source == "DataBase" \
                and event.destination == "FileSystem" \
                and event.operation == "read_request" \
                and event.parameters is not None \
                and event.data is None:
            authorized = True

        # запрос на чтение данных из файловой систему
        if event.source == "FileSystem" \
                and event.destination == "FileSystem" \
                and event.operation == "read_response" \
                and event.parameters is not None \
                and event.data is None:
            authorized = True



        if authorized is False:
            print(f"[{self.__class__.__name__}] событие не разрешено политиками безопасности")
        return authorized

    # выполнение разрешённого запроса
    # метод должен вызываться только после проверки политик безопасности
    def _proceed(self, event):
        print(f'[{self.__class__.__name__}] отправляем запрос {event}')
        try:
            # найдём очередь получателя события
            dst_q: Queue = self._entity_queues[event.destination]
            # и положим запрос в эту очередь
            dst_q.put(event)
        except  Exception as e:
            # например, запрос пришёл от или для неизвестной сущности
            print(f"[{self.__class__.__name__}] ошибка выполнение запроса {e}")

    # основной код работы монитора безопасности    
    def run(self):
        print(f'[{self.__class__.__name__}] старт')

        # в цикле проверяет наличие новых событий, 
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим, 
                # чтобы можно было завершить работу монитора, 
                # не дожидаясь нового сообщения
                event = self._events_q.get_nowait()
                # сюда попадаем только в случае получение события,
                # теперь нужно проверить политики безопасности
                authorized = self._check_policies(event)
                if authorized:
                    # если политиками запрос авторизован - выполняем
                    self._proceed(event)
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")
            self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    # запрос на остановку работы монитора безопасности для завершения работы
    # может вызываться вне процесса монитора
    def stop(self):
        # поскольку монитор работает в отдельном процессе,
        # запрос помещается в очередь, которая проверяется из процесса монитора
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность DataProcessor

Взаимодействует с базой данных для обработки данных и записи результатов обработки.

In [108]:
from multiprocessing import Queue, Process


class DataProcessor(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        # очередь управляющих команд (например, для остановки)
        self._control_q = Queue()
        self._force_quit = False

    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue

    @staticmethod
    def _sample_data():
        return {
            "id": "12345",
            "client": "John Doe",            
            "order_details": {
                "name": "cisplatin",
                "quantity": 12,
                "payment_id": "aaa-bbb-cccc",
                "total": 10000
            }
        }

    @staticmethod
    def encrypt_field(src, field_name) -> dict:
        # исключительно для демонстрационных целей, чтобы не усложнять,
        # используем собственную реализацию простейшего перестановочного шифра
        # этот код нельзя использовать в реальных системах ни при каких условиях!
        key = 3
        plain_text = src[field_name]
        cipher = ""
        for char in plain_text:
            if char not in ' ,.':
                cipher +=  chr(ord('A') + (ord(char) - ord('A') + key) % 26)
            else:
                cipher += char
        src[field_name] = cipher
        return src


    @staticmethod
    def decrypt_field(src, field_name) -> dict:
        key = 3
        cipher = src[field_name]
        plain_text = ""
        for char in cipher:
            if char not in ' ,.':
                plain_text +=  chr(ord('A') + (ord(char) - ord('A') - key) % 26)
            else:
                plain_textr += char
        src[field_name] = plain_text
        return src

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        print(f'[{self.__class__.__name__}] отправляем тестовый запрос')

        # запишем тестовые данные
        data = self.encrypt_field(self._sample_data(), "client")
        print(f'[{self.__class__.__name__}] записываем данные {data}')
        event = Event(source=self.__class__.__name__,
                      destination='DataBase',
                      operation='write_request',
                      parameters='table_orders',
                      data=data
                      )
        

        self.monitor_queue.put(event)
        print(f'[{self.__class__.__name__}] завершение работы')

    def stop(self):
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность DataBase

Эта сущность предоставляет интерфейс для работы с данными, абстрагируя от уровня записи в файловую систему.

In [109]:
from multiprocessing import Queue, Process
from time import sleep


class DataBase(Process):

    def __init__(self, monitor_queue: Queue):
        # конструктор аналогичный
        super().__init__()
        self.monitor_queue = monitor_queue
        self._own_queue = Queue()
        self._control_q = Queue()
        self._force_quit = False

    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        while self._force_quit is False:
            try:
                event: Event = self._own_queue.get_nowait()
                if event.operation == "create":
                    print(f"[{self.__class__.__name__}] {event.source} захотел создать данные {event.parameters}")
                elif event.operation == "update":
                    print(f"[{self.__class__.__name__}] {event.source} захотел обновить данные {event.parameters}")
                elif event.operation == "read":
                    print(f"[{self.__class__.__name__}] {event.source} захотел прочитать данные {event.parameters}")
            except Empty:                
                sleep(0.2)
                self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    def stop(self):
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность FileSystem

In [110]:
from multiprocessing import Queue, Process
from time import sleep


class FileSystem(Process):

    def __init__(self, monitor_queue: Queue):
        # конструктор аналогичный
        super().__init__()
        self.monitor_queue = monitor_queue
        self._own_queue = Queue()
        self._control_q = Queue()  # очередь управляющих команд (например, для остановки)
        self._force_quit = False

    def entity_queue(self):
        return self._own_queue

    def write_file(self, details: Event):
        print(f"[{self.__class__.__name__}] {details.source} записываем файл {details.parameters} с содержимым {details.data}")

    def read_file(self, details: Event) -> str or None:
        print(f"[{self.__class__.__name__}] {details.source} читаем файл {details.parameters}")
        return None

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')        
        while self._force_quit is False:
            try:
                event: Event = self._own_queue.get_nowait()
                if event.operation == "write":
                    print(f"[{self.__class__.__name__}] {event.source} запрос на запись файла {event.parameters} с содержимым {event.data}")
                    self.write_file(event)
                    break
                elif event.operation == "read":
                    print(f"[{self.__class__.__name__}] {event.source} запрос на чтение файла {event.parameters}")
                    self.read_file(event)
                    # TODO: отправить содержимое файла запрашивающему
                    break
                elif event.operation == "delete":
                    # удаление файла, пока не поддерживается
                    print(f"[{self.__class__.__name__}] {event.source} запрос на удаление файла {event.parameters}")
                    break
            except Empty:
                sleep(0.2)
                self._check_control_q()                
        print(f'[{self.__class__.__name__}] завершение работы')

    def stop(self):
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Инициализируем монитор и сущности

In [111]:
monitor = Monitor(monitor_events_queue)
data_processor = DataProcessor(monitor_events_queue)
data_base = DataBase(monitor_events_queue)
file_system = FileSystem(monitor_events_queue)

регистрируем очереди сущностей в мониторе

In [112]:
monitor.add_entity_queue(data_processor.__class__.__name__, data_processor.entity_queue())
monitor.add_entity_queue(data_base.__class__.__name__, data_base.entity_queue())
monitor.add_entity_queue(file_system.__class__.__name__, file_system.entity_queue())

[Monitor] регистрируем сущность DataProcessor
[Monitor] регистрируем сущность DataBase
[Monitor] регистрируем сущность FileSystem


### Запускаем всё

Ожидаемая последовательность событий

![Диаграмма последовательности вызовов](https://www.plantuml.com/plantuml/png/SoWkIImgAStDuU8g038oapCB4lDA5CBpYx9JYnnHn7ppyp9Byekmg7FYue9gKD2rWwdJrhPIkDXuOT-5ZHUxBpPTs75XgyA5BHTsN-o7Qu8gHD9ZfN1X_yK6e9vD5xO2wLqNDW05WDpT5tQ0ZTY5f0TcZAukfZtxBZR1lP2ON99PbbYI2hSMIgCP0J80QEM62nj1ka5MevXwiP-BYyiXDIy565G0)

In [113]:
monitor.start()
file_system.start()
data_base.start()
data_processor.start()
sleep(2)

[Monitor] старт
[FileSystem] старт
[DataBase] старт
[DataProcessor] старт
[DataProcessor] отправляем тестовый запрос
[DataProcessor] записываем данные {'id': '12345', 'client': 'MXQW GXN', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}}
[DataProcessor] завершение работы
[Monitor] обрабатываем событие Event(source='DataProcessor', destination='DataBase', operation='write_request', parameters='table_orders', data={'id': '12345', 'client': 'MXQW GXN', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}})
[Monitor] отправляем запрос Event(source='DataProcessor', destination='DataBase', operation='write_request', parameters='table_orders', data={'id': '12345', 'client': 'MXQW GXN', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}})


### Теперь останавливаем

Сначала отправим запросы на останов процессов

In [114]:
monitor.stop()
data_processor.stop()
data_base.stop()
file_system.stop()


Теперь подождём завершения их работы

In [115]:
data_processor.join()
data_base.join()
file_system.join()
monitor.join()

[FileSystem] проверяем запрос ControlEvent(operation='stop')
[FileSystem] завершение работы
[DataBase] проверяем запрос ControlEvent(operation='stop')
[DataBase] завершение работы
[Monitor] проверяем запрос ControlEvent(operation='stop')
[Monitor] завершение работы


## Заключение

В этом блокноте продемонстрирован подход FLASK: тотальный контроль взаимодействия сущностей с помощью монитора безопасности и простейших политик.

Пример не так прост, как может показаться: используется многопроцессная реализация - каждая сущность запускается в отдельном процессе, а передача данных осуществляется через специализированные очереди. После запуска сущностей доступ к их памяти становится защищён операционной системой. Это можно с натяжкой рассматривать как имитацию MILS функциональности.

## Упражнения

Уровень "Новичок"

- в коде WorkerA измените текст передаваемого сообщения и выполните все ячейки. Убедитесь, что монитор безопасности заблокировал сообщение, как нарушающее политику безопасности
- добавьте текстовое сообщение в мониторе после успешной проверки политик безопасности
- измените политики безопасности так, чтобы любой текст был допустим

Уровень "Шутки закончились"

- измените код сущностей так, чтобы передача сообщений между WorkerA и WorkerB была двунаправленной: после получения сообщения от WorkerA WorkerB посылает ответное сообщение отправителю (например, say "done"), WorkerA должен получить сообщение и выполнить команду. С помощью plantuml.com нарисуйте диаграмму последовательности для этого сценария и вставьте в блокнот.

Уровень "Полубог"

_Возможно, для задач этого уровня сложности имеет смысл перенести разработку кода в VS Code или PyCharm._

- добавьте третью сущность (WorkerC), которая будет периодически (например, раз в минуту) запрашивать внешнюю информацию - например, курс валют с сайта ЦБ РФ (https://www.cbr-xml-daily.ru/latest.js) и отправлять WorkerA, а WorkerA в случае превышения заданного порога будет уведомлять WorkerB о событии. Очевидно, для этого сценария нужно доработать политики безопасности.